# Ressources

In [36]:
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.cluster import adjusted_rand_score
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans


# fix pour plotly express et Visual Studio Code
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Import
## Data

In [37]:
df_tsne = pd.read_csv('ml/df_tsne.csv')
df_clients = pd.read_csv('df_clients.csv')



Variables discriminantes

In [38]:
X = ['Dernière commande (Jours)', 'Nombre_commandes', 'payment_value_total', 'Nb de moyens de paiement', 'review_score_avg', 'Nombre_produits_achetes']

## Modèle K-means

In [39]:
kmeans_ss = pickle.load(open('ml/kmeans_ss.pkl', 'rb'))
kmeans_pca = pickle.load(open('ml/kmeans_pca.pkl', 'rb'))
kmeans_seg = pickle.load(open('ml/kmeans_seg.pkl', 'rb'))

# Maintenance
## Création des périodes

In [40]:
df_clients

,customer_unique_id,review_score_avg,Délai review_avg,payment_sequential_avg,payment_installments_avg,product_name_lenght_avg,product_description_lenght_avg,product_photos_qty_avg,product_weight_g_avg,product_volume_cm^3_avg,...,customer_state,Nb de moyens de paiement,Dernière commande(Date),dont frais de livraison,Dernière commande (Jours),payment_value_total,Nombre_produits_achetes,Nombre_commandes,payment_avg,frais de livraison_avg
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,4.572778,1.0,8.0,60.0,236.0,1.0,1500.0,7616.0,...,SP,1,2018-05-10 10:56:27,12.00,115,141.90,1,1,141.90,12.00
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4.0,0.937095,1.0,1.0,56.0,635.0,1.0,375.0,5148.0,...,SP,1,2018-05-07 11:11:27,8.29,118,27.19,1,1,27.19,8.29
2,0000f46a3911fa3c0805444483337064,3.0,1.055370,1.0,8.0,49.0,177.0,3.0,1500.0,43750.0,...,SC,1,2017-03-10 21:05:03,17.22,541,86.22,1,1,86.22,17.22
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,1.012396,1.0,4.0,43.0,1741.0,5.0,150.0,1045.0,...,PA,1,2017-10-12 20:29:41,17.63,325,43.62,1,1,43.62,17.63
4,0004aac84e0df4da2b147fca70cf8255,5.0,4.937211,1.0,6.0,58.0,794.0,3.0,6050.0,528.0,...,SP,1,2017-11-14 19:45:42,16.89,292,196.89,1,1,196.89,16.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97231,fffcf5a5ff07b0908bd4e2dbc735a684,5.0,4.472280,1.0,10.0,54.0,478.5,1.0,30000.0,251625.0,...,PE,1,2017-06-08 21:00:36,497.42,451,2067.42,2,1,2067.42,497.42
97232,fffea47cd6d3cc0a88bd621562a9d061,4.0,5.494433,1.0,1.0,37.0,506.0,2.0,1720.0,32076.0,...,BA,1,2017-12-10 20:07:56,19.69,266,84.58,1,1,84.58,19.69
97233,ffff371b4d645b6ecea244b27531430a,5.0,2.627326,1.0,1.0,57.0,716.0,5.0,600.0,3315.0,...,MT,1,2017-02-07 15:49:16,22.56,572,112.46,1,1,112.46,22.56
97234,ffff5962728ec6157033ef9805bacc48,5.0,2.636343,1.0,5.0,35.0,591.0,3.0,444.0,4480.0,...,ES,1,2018-05-02 15:17:41,18.69,123,133.69,1,1,133.69,18.69


In [41]:
# Type date
df_clients['Dernière commande(Date)'] = pd.to_datetime(df_clients['Dernière commande(Date)'])
# Piste possible
df_clients['Année'] = df_clients['Dernière commande(Date)'].dt.year
df_clients['Mois'] = df_clients['Dernière commande(Date)'].dt.month

df_clients

,customer_unique_id,review_score_avg,Délai review_avg,payment_sequential_avg,payment_installments_avg,product_name_lenght_avg,product_description_lenght_avg,product_photos_qty_avg,product_weight_g_avg,product_volume_cm^3_avg,...,Dernière commande(Date),dont frais de livraison,Dernière commande (Jours),payment_value_total,Nombre_produits_achetes,Nombre_commandes,payment_avg,frais de livraison_avg,Année,Mois
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,4.572778,1.0,8.0,60.0,236.0,1.0,1500.0,7616.0,...,2018-05-10 10:56:27,12.00,115,141.90,1,1,141.90,12.00,2018,5
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4.0,0.937095,1.0,1.0,56.0,635.0,1.0,375.0,5148.0,...,2018-05-07 11:11:27,8.29,118,27.19,1,1,27.19,8.29,2018,5
2,0000f46a3911fa3c0805444483337064,3.0,1.055370,1.0,8.0,49.0,177.0,3.0,1500.0,43750.0,...,2017-03-10 21:05:03,17.22,541,86.22,1,1,86.22,17.22,2017,3
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,1.012396,1.0,4.0,43.0,1741.0,5.0,150.0,1045.0,...,2017-10-12 20:29:41,17.63,325,43.62,1,1,43.62,17.63,2017,10
4,0004aac84e0df4da2b147fca70cf8255,5.0,4.937211,1.0,6.0,58.0,794.0,3.0,6050.0,528.0,...,2017-11-14 19:45:42,16.89,292,196.89,1,1,196.89,16.89,2017,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97231,fffcf5a5ff07b0908bd4e2dbc735a684,5.0,4.472280,1.0,10.0,54.0,478.5,1.0,30000.0,251625.0,...,2017-06-08 21:00:36,497.42,451,2067.42,2,1,2067.42,497.42,2017,6
97232,fffea47cd6d3cc0a88bd621562a9d061,4.0,5.494433,1.0,1.0,37.0,506.0,2.0,1720.0,32076.0,...,2017-12-10 20:07:56,19.69,266,84.58,1,1,84.58,19.69,2017,12
97233,ffff371b4d645b6ecea244b27531430a,5.0,2.627326,1.0,1.0,57.0,716.0,5.0,600.0,3315.0,...,2017-02-07 15:49:16,22.56,572,112.46,1,1,112.46,22.56,2017,2
97234,ffff5962728ec6157033ef9805bacc48,5.0,2.636343,1.0,5.0,35.0,591.0,3.0,444.0,4480.0,...,2018-05-02 15:17:41,18.69,123,133.69,1,1,133.69,18.69,2018,5


In [42]:
df_tsne[['tsne1', 'tsne2', 'tsne3']] = df_tsne[['tsne1', 'tsne2', 'tsne3']].astype("float32")
df_tsne.dtypes

customer_unique_id                 object
review_score_avg                  float64
Délai review_avg                  float64
payment_sequential_avg            float64
payment_installments_avg          float64
product_name_lenght_avg           float64
product_description_lenght_avg    float64
product_photos_qty_avg            float64
product_weight_g_avg              float64
product_volume_cm^3_avg           float64
Délai de livraison_avg            float64
customer_city                      object
customer_state                     object
Nb de moyens de paiement            int64
Dernière commande(Date)            object
dont frais de livraison           float64
Dernière commande (Jours)           int64
payment_value_total               float64
Nombre_produits_achetes             int64
Nombre_commandes                    int64
payment_avg                       float64
frais de livraison_avg            float64
cluster_rfm                        object
cluster_ss                        

## Calcul ARI 

Simulation en <span style="color: #00B0F0">marche arrière</span>
### Scaling

In [43]:
def calc_maintenance_ss(df_clients, kmeans, date):
    start = df_clients['Dernière commande (Jours)'].max() # la date max en jour est celle qui date le plus.
    df = df_clients.loc[df_clients['Dernière commande (Jours)'] <=(start-date)] # on va, à chaque boucle, rajouter 15 jours de plus (voir cellule plus bas)
    df = df[X] # notre segmentation
    ss = StandardScaler() 
    scaled_features_seg = ss.fit_transform(df[X]) # on standardise
    
    kmeans2 = KMeans(n_clusters=4).fit(scaled_features_seg)  # on initialise notre nouveau kmeans.
    
   
    C1_init = kmeans.predict(scaled_features_seg)  # avec l'ancien kmeans
    C1_pred = kmeans2.predict(scaled_features_seg) # avec le nouveau
    
    # The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
    ari_p = adjusted_rand_score(C1_init, C1_pred)
    return ari_p

In [44]:
dates = np.arange(30,720,15)
ari_s = []

date_line = np.arange(0,705,1)
line_ari = np.linspace(0.8,0.8,705)

for date in dates:
    ari_p = calc_maintenance_ss(df_clients, kmeans_ss, date)
    ari_s.append(ari_p)
    
fig = px.line(x=dates, y=ari_s, title="Stabilité temporelle Kmeans Standardise", labels={"x":"nombre de jours depuis le début",
                                                                            "y": "Score ARI"},)

fig.add_trace(go.Scatter(x=date_line, y=line_ari, mode="lines", name="score acceptable"))
fig.show()

### T-SNE

In [45]:
def calc_maintenance(df_clients, kmeans, date):
    start = df_clients['Dernière commande (Jours)'].max() # la date max en jour est celle qui date le plus.
    df = df_clients.loc[df_clients['Dernière commande (Jours)'] <=(start-date)] # on va, à chaque boucle, rajouter 15 jours de plus (voir cellule plus bas)
    df = df[['tsne1', 'tsne2', 'tsne3']] # notre segmentation
    
    kmeans2 = KMeans(n_clusters=4).fit(df)  # on initialise notre nouveau kmeans.
    
    C1_init = kmeans.predict(df)  # avec l'ancien kmeans
    C1_pred = kmeans2.predict(df) # avec le nouveau
    
    # The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
    ari_p = adjusted_rand_score(C1_init, C1_pred)
    return ari_p

In [46]:
dates = np.arange(30,720,15)
ari_s = [] 

for date in tqdm(dates):
    ari_p = calc_maintenance(df_tsne, kmeans_seg, date)
    ari_s.append(ari_p)
    
fig = px.line(x=dates, y=ari_s, title="Stabilité temporelle Kmeans T-SNE", labels={"x":"nombre de jours depuis le début",
                                                                            "y": "Score ARI"},)

fig.add_trace(go.Scatter(x=date_line, y=line_ari, mode="lines", name="score acceptable"))
fig.show()

100%|██████████| 46/46 [00:23<00:00,  1.94it/s]


### PCA

In [47]:
def calc_maintenance_pca(df_clients, kmeans, date):
    start = df_clients['Dernière commande (Jours)'].max() # la date max en jour est celle qui date le plus.
    df = df_clients.loc[df_clients['Dernière commande (Jours)'] <=(start-date)] # on va, à chaque boucle, rajouter 15 jours de plus (voir cellule plus bas)
    df = df[X] # notre segmentation
    ss = StandardScaler() 
    scaled_features_seg = ss.fit_transform(df[X]) # on standardise
    pca_seg = PCA(n_components=3) # 3 composants. On compare avec ce qu'on a fait avant.. On refait donc les mêmes étapes   
    scaled_features_seg = pca_seg.fit_transform(scaled_features_seg) # pca
    
    kmeans2 = KMeans(n_clusters=4).fit(scaled_features_seg)  # on initialise notre nouveau kmeans.
    
   
    C1_init = kmeans.predict(scaled_features_seg)  # avec l'ancien kmeans
    C1_pred = kmeans2.predict(scaled_features_seg) # avec le nouveau
    
    # The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
    ari_p = adjusted_rand_score(C1_init, C1_pred)
    return ari_p

In [48]:
dates = np.arange(30,720,15)
ari_s = []

for date in dates:
    ari_p = calc_maintenance_pca(df_clients, kmeans_pca, date)
    ari_s.append(ari_p)
    
fig = px.line(x=dates, y=ari_s, title="Stabilité temporelle Kmeans PCA", labels={"x":"nombre de jours depuis le début",
                                                                            "y": "Score ARI"},)

fig.add_trace(go.Scatter(x=date_line, y=line_ari, mode="lines", name="score acceptable"))
fig.show()